In [1]:
import numpy as np
import pandas as pd

### Split ratings into train and test sets

In [3]:
movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0, encoding='iso-8859-1')
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0, encoding='iso-8859-1')
movielens_test.shape

(2668, 11)

### Prepare datastructures for estimation. Movie*User matrix. 

In [4]:
ratings_mtx_df = movielens_train.pivot_table(values='rating',index='movie_id',columns='user_id')
ratings_mtx_df.fillna(0, inplace=True)
ratings_mtx_df.head()

user_id,5,8,10,13,15,18,19,24,25,26,...,6016,6018,6019,6021,6022,6025,6030,6031,6036,6037
movie_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
ratings_mtx_df.shape

(2477, 3388)

In [5]:
ratings_mtx = ratings_mtx_df.as_matrix()

In [7]:
ratings_mtx

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [6]:
ratings_test_mtx_df = movielens_test.pivot_table(values='rating',index='movie_id',columns='user_id')
ratings_test_mtx_df.fillna(0, inplace=True)
ratings_test_mtx_df.head()
ratings_test_mtx = ratings_test_mtx_df.as_matrix()

In [7]:
ratings_test_mtx

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  3.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

### create a logical matrix (matrix that represents whether a rating was made, or not)

In [8]:
did_rate = (ratings_mtx != 0) * 1

In [9]:
did_rate

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### A function that normalizes a dataset

In [10]:
def normalize_ratings(ratings, did_rate):
    num_movies = ratings.shape[0]
    
    ratings_mean = np.zeros(shape = (num_movies, 1))
    ratings_norm = np.zeros(shape = ratings.shape)
    
    for i in range(num_movies): 
        # Get all the indexes where there is a 1
        idx = np.where(did_rate[i] == 1)[0]
        #  Calculate mean rating of ith movie only from user's that gave a rating
        ratings_mean[i] = np.mean(ratings[i, idx])
        ratings_norm[i, idx] = ratings[i, idx] - ratings_mean[i]
    
    return ratings_norm, ratings_mean

In [11]:
# Normalize ratings
ratings, ratings_mean = normalize_ratings(ratings_mtx, did_rate)

In [12]:
ratings

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [13]:
ratings_mean.size

1934

In [14]:
# Update some key variables now
num_users = ratings.shape[1]
num_movies = ratings.shape[0]
num_features = 3

In [15]:
num_users,num_movies,num_features

(2170, 1934, 3)

### Initialize Parameters theta (user_prefs), X (movie_features) 

In [16]:
movie_features = np.random.randn( num_movies, num_features )
user_prefs = np.random.randn( num_users, num_features )
initial_X_and_theta = np.r_[movie_features.T.flatten(), user_prefs.T.flatten()]

In [17]:
movie_features.size

5802

In [18]:
user_prefs.size

6510

In [19]:
initial_X_and_theta

array([-0.42329967,  0.10054861, -0.23096118, ...,  0.63065207,
       -0.55369861, -1.06933575])

In [20]:
def unroll_params(X_and_theta, num_users, num_movies, num_features):
	# Retrieve the X and theta matrixes from X_and_theta, based on their dimensions (num_features, num_movies, num_movies)
	# --------------------------------------------------------------------------------------------------------------
	# Get the first 30 (10 * 3) rows in the 48 X 1 column vector
	first_30 = X_and_theta[:num_movies * num_features]
	# Reshape this column vector into a 10 X 3 matrix
	X = first_30.reshape((num_features, num_movies)).transpose()
	# Get the rest of the 18 the numbers, after the first 30
	last_18 = X_and_theta[num_movies * num_features:]
	# Reshape this column vector into a 6 X 3 matrix
	theta = last_18.reshape(num_features, num_users ).transpose()
	return X, theta

In [21]:
X, theta = unroll_params(initial_X_and_theta, num_users, num_movies, num_features)

In [22]:
X.size, theta.size

(5802, 6510)

In [23]:
theta

array([[ 0.55320995,  1.25416231, -0.56472285],
       [-1.29254423, -0.04175409, -0.17773628],
       [-0.59595639, -0.67803479,  0.51458217],
       ..., 
       [ 0.41446185, -1.08024399,  0.63065207],
       [ 1.50768209, -0.35633   , -0.55369861],
       [-2.12220639,  0.28964042, -1.06933575]])

In [26]:
def calculate_gradient(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
	X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)
	
	# we multiply by did_rate because we only want to consider observations for which a rating was given
	difference = X.dot( theta.T ) * did_rate - ratings
	X_grad = difference.dot( theta ) + reg_param * X
	theta_grad = difference.T.dot( X ) + reg_param * theta
	
	# wrap the gradients back into a column vector 
	return np.r_[X_grad.T.flatten(), theta_grad.T.flatten()]

In [27]:
def calculate_cost(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
	X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)
	
	# we multiply (element-wise) by did_rate because we only want to consider observations for which a rating was given
	cost = np.sum( (X.dot( theta.T ) * did_rate - ratings) ** 2 ) / 2
	# '**' means an element-wise power
	regularization = (reg_param / 2) * (np.sum( theta**2 ) + np.sum(X**2))
	return cost + regularization

In [30]:
# import these for advanced optimizations (like gradient descent)

from scipy import optimize

In [44]:
# regularization paramater

# perform gradient descent, find the minimum cost (sum of squared errors) and optimal values of X (movie_features) and Theta (user_prefs)

reg_param = 20

minimized_cost_and_optimal_params = optimize.fmin_cg(calculate_cost, fprime=calculate_gradient, x0=initial_X_and_theta, 								args=(ratings, did_rate, num_users, num_movies, num_features, reg_param), 								maxiter=100, disp=True, full_output=True ) 

cost, optimal_movie_features_and_user_prefs = minimized_cost_and_optimal_params[1], minimized_cost_and_optimal_params[0]


# unroll once again

movie_features, user_prefs = unroll_params(optimal_movie_features_and_user_prefs, num_users, num_movies, num_features)

         Current function value: 2151.468533
         Iterations: 2
         Function evaluations: 19
         Gradient evaluations: 7


### Make some predictions (movie recommendations). Dot product 

In [45]:
all_predictions = movie_features.dot( user_prefs.T )
all_predictions

array([[-0.0035341 , -0.01174585, -0.00293301, ...,  0.00951802,
        -0.00721105, -0.00582644],
       [ 0.00216216,  0.0008276 , -0.00056165, ..., -0.0021407 ,
         0.00325097, -0.00085147],
       [ 0.00550022,  0.0032277 , -0.00185473, ..., -0.00803746,
        -0.00230468, -0.00190277],
       ..., 
       [ 0.00189344, -0.00085902, -0.00053845, ...,  0.00028868,
         0.00947541, -0.00151429],
       [-0.00282773, -0.00092786,  0.00055061, ...,  0.00215388,
        -0.00731177,  0.00107298],
       [ 0.00452678, -0.00533913, -0.00382798, ..., -0.00045944,
         0.00510951, -0.00671363]])

In [46]:
# add back the ratings_mean column vector to my (our) predictions
all_predictions = all_predictions + ratings_mean
all_predictions

array([[ 4.17293649,  4.16472474,  4.17353758, ...,  4.18598861,
         4.16925954,  4.17064415],
       [ 2.66882882,  2.66749427,  2.66610501, ...,  2.66452597,
         2.66991763,  2.66581519],
       [ 3.00550022,  3.0032277 ,  2.99814527, ...,  2.99196254,
         2.99769532,  2.99809723],
       ..., 
       [ 4.00189344,  3.99914098,  3.99946155, ...,  4.00028868,
         4.00947541,  3.99848571],
       [ 3.99717227,  3.99907214,  4.00055061, ...,  4.00215388,
         3.99268823,  4.00107298],
       [ 4.00452678,  3.99466087,  3.99617202, ...,  3.99954056,
         4.00510951,  3.99328637]])

In [47]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [48]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    cnt_users = ratings_test_mtx.shape[1]
    cnt_movies = ratings_test_mtx.shape[0]
    for u in range(0, cnt_movies-1):
        for i in range(0, cnt_users-1):
            estimated = np.array([estimate_f(u,i)])
            real = np.array([ratings_test_mtx[u][i]])
        
    return compute_rmse(estimated, real)    

In [49]:
def estimate(movie_id, user_id):
    return all_predictions[movie_id][user_id]

In [50]:
print('RMSE for engine: %s' % evaluate(estimate))

RMSE for engine: 1.98985149154
